In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('resources/exoplanet_data.csv')
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
#This is a first pass at feature selection. I essentially only eliminated columns that I knew for sure should not be included
feature_select = df[['koi_period', 'koi_period_err1', 'koi_period_err2',
                     'koi_impact', 'koi_impact_err1', 'koi_impact_err2', 
                     'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 
                     'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
                     'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
                     'koi_teq', 'koi_model_snr',
                     'koi_steff', 'koi_steff_err1', 'koi_steff_err2',
                     'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
                     'koi_srad', 'koi_srad_err1', 'koi_srad_err2',
                     'ra', 'dec', 'koi_kepmag',
                     'koi_disposition']]

In [4]:
feature_select.head()

,koi_period,koi_period_err1,koi_period_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,koi_disposition
0,54.418383,2.479000e-04,-2.479000e-04,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,CONFIRMED
1,19.899140,1.490000e-05,-1.490000e-05,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,...,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,FALSE POSITIVE
2,1.736952,2.630000e-07,-2.630000e-07,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,...,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,FALSE POSITIVE
3,2.525592,3.760000e-06,-3.760000e-06,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,...,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,CONFIRMED
4,4.134435,1.050000e-05,-1.050000e-05,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,...,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714,CONFIRMED


In [5]:
dataset = feature_select[feature_select['koi_disposition'] != 'CANDIDATE']
gameset = feature_select[feature_select['koi_disposition'] == 'CANDIDATE']

In [6]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [8]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_mms = mms.fit_transform(X_train)
X_test_mms = mms.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression
log_cla = LogisticRegression(random_state = 0)
log_cla.fit(X_train_mms, y_train)
log_pred = log_cla.predict(X_test_mms)

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, log_pred)
print(cm)
accuracy_score(y_test, log_pred)

[[323 124]
 [ 71 808]]


0.8529411764705882

In [11]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric = 'minkowski', p = 2)
knn.fit(X_train_mms,y_train)
knn_pred = knn.predict(X_test_mms)

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, knn_pred)
print(cm)
accuracy_score(y_test, knn_pred)

[[374  73]
 [112 767]]


0.8604826546003017